In [ ]:
import tensorflow as tf
import pandas as pd
import glob
import cv2
import numpy as np

In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
model = InceptionV3(include_top=False, weights='imagenet', input_shape=(299,299,3), pooling='avg')

In [ ]:
df = pd.read_csv('movie_genre_matrix.csv')

In [ ]:
def png_to_numpy(movie_name):
    movie_file_strings = glob.glob(f'trailer_test/{movie_name}*.png')
    return [(movie_name, cv2.imread(movie)) for movie in movie_file_strings]
    

In [ ]:
def create_movie_df(list_of_movies):
    
    movie_titles = []
    frames = []
    for movie in list_of_movies:
        for movie_title, frame in png_to_numpy(movie):
            movie_titles.append(movie_title)
            frames.append(frame)
    
    dict_of_numpys = {'movie_title': movie_titles, 'frames': frames}
    df_numpy = pd.DataFrame(dict_of_numpys)
    df_numpy = pd.merge(df_numpy, df, on='movie_title')
    return df_numpy
        

In [ ]:
def get_deep_features(frame):
    prediction = model.predict(frame.reshape((-1, 299, 299, 3)))
    return prediction

In [ ]:
get_deep_features_for_multiple_frames = lambda frames: np.vstack([get_deep_features(frame) for frame in frames])

In [ ]:
labels_and_featuers = pd.concat([movie_train_no_frames, predictions_df], axis=1)

In [ ]:
def split_by_movie(df, test_percentage):
    movie_titles = df['movie_title'].unique()
    test_titles = np.random.choice(movie_titles, size=int(len(movie_titles)//test_percentage), replace=False)

    test_df = df[df['movie_title'].isin(test_titles)]
    train_df = df[~df['movie_title'].isin(test_titles)]
    
    genres = df.columns[1:27]
    features_movie_title = df.columns.drop(genres)
    
    
    #X_train, X_test, y_train, y_test 
    return train_df[features_movie_title], test_df[features_movie_title], train_df[genres], test_df[genres]

In [ ]:
X_train, X_test, y_train, y_test = split_by_movie(labels_and_featuers, 10)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report